### Instalação das dependencias


In [ ]:
!pip install feedparser beautifulsoup4
!pip install transformers sentence-transformers
!pip install bitsandbytes accelerate
!pip install faiss-cpu -q

In [ ]:
import re
import gc
import numpy as np
import pandas as pd
import torch
import faiss
import feedparser
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

### Definição das Fontes RSS

In [ ]:
RSS_FEEDS = [
    # --- Notícias Locais (Paraíba) ---
    "https://www.jornaldaparaiba.com.br/rss",
    "https://www.clickpb.com.br/rss",
    "https://www.paraibaonline.com.br/rss",
    "https://www.pbagora.com.br/rss",
    "https://f5online.com.br/feed/",
    "https://www.portalt5.com.br/noticias/feed.rss",
    "https://www.polemicaparaiba.com.br/feed/",
    "https://www.maispb.com.br/feed",                 # MaisPB
    "https://www.wscom.com.br/feed/",                 # WSCOM Notícias

    # --- Regionais (Nordeste) ---
    "https://www.diariodonordeste.com.br/rss.xml",    # Diário do Nordeste (CE)
    "https://www.op9.com.br/feed/",                   # OP9 (Nordeste)
    "https://www.cbnrecife.com/rss",                  # CBN Recife
    "https://www.jornaldocomercio.com/_conteudo/feed",# JC Online (PE)
    "https://www.atarde.com.br/rss",                  # A Tarde (BA)

    # --- Nacionais (Portais Diversos) ---
    "http://rss.uol.com.br/feed/noticias.xml",        # UOL Notícias
    "https://noticias.r7.com/feed.xml",               # R7 Notícias
    "https://www.estadao.com.br/rss/ultimas.xml",     # Estadão Últimas
    "http://feeds.folha.uol.com.br/emcimadahora/rss091.xml", # Folha em Cima da Hora
    "https://valor.globo.com/rss/",                   # Valor Econômico
    "https://www.infomoney.com.br/feed/",             # InfoMoney
    "https://oglobo.globo.com/rss.xml",               # O Globo
    "https://www.cartacapital.com.br/feed/",          # Carta Capital
    "https://www.conjur.com.br/index.php/feed",       # Consultor Jurídico (Direito/Política)

    # --- Nacionais (G1, por editoria) ---
    "https://g1.globo.com/rss/g1/brasil/",
    "https://g1.globo.com/rss/g1/carros/",
    "https://g1.globo.com/rss/g1/ciencia-e-saude/",
    "https://g1.globo.com/rss/g1/economia/",
    "https://g1.globo.com/rss/g1/educacao/",
    "https://g1.globo.com/rss/g1/mundo/",
    "https://g1.globo.com/rss/g1/politica/",
    "https://g1.globo.com/rss/g1/tecnologia/",
    "https://g1.globo.com/rss/g1/pb/paraiba/",
    "https://g1.globo.com/rss/g1/pop-arte/",           # Cultura
    "https://g1.globo.com/rss/g1/natureza/",           # Meio Ambiente
    "https://g1.globo.com/rss/g1/turismo-e-viagem/",   # Viagem
    "https://g1.globo.com/rss/g1/cultura/",            # Artes e cultura
    "https://g1.globo.com/rss/g1/fato-ou-fake/",       # Checagem de fatos
    # Google News - consultas personalizadas
    "https://news.google.com/rss/search?q=Paraíba&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Brasil&hl=pt-BR&gl=BR&ceid=BR:pt-419",

    # Capitais brasileiras
    "https://news.google.com/rss/search?q=Rio+Branco&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Maceió&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Macapá&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Manaus&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Salvador&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Fortaleza&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Brasília&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Vitória&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Goiânia&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=São+Luís&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Cuiabá&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Campo+Grande&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Belo+Horizonte&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Belém&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=João+Pessoa&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Curitiba&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Recife&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Teresina&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Rio+de+Janeiro&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Natal&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Porto+Alegre&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Porto+Velho&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Boa+Vista&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Florianópolis&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=São+Paulo&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Aracaju&hl=pt-BR&gl=BR&ceid=BR:pt-419",
    "https://news.google.com/rss/search?q=Palmas&hl=pt-BR&gl=BR&ceid=BR:pt-419"

]


## Coleta e Pré-processamento dos Dados

In [ ]:
def fetch_and_clean_news(feed_urls):
    articles = []
    for url in feed_urls:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            soup = BeautifulSoup(entry.get('summary', ''), 'html.parser')
            text_content = re.sub(r'\s+', ' ', soup.get_text()).strip()

            articles.append({
                'fonte': feed.feed.get('title', 'N/A'),
                'titulo': entry.title,
                'link': entry.link,
                'descricao': text_content,
                'data': entry.get('published', 'N/A')
            })
    return pd.DataFrame(articles)

df_feeds = fetch_and_clean_news(RSS_FEEDS)


In [ ]:
print(f"Total de notícias coletadas: {len(df_feeds)}")
df_feeds.head()

In [ ]:
df_feeds['content_for_embedding'] = df_feeds['titulo'] + ". " + df_feeds['descricao']

## Vetorização do Conteúdo e Indexação

In [ ]:
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device='cuda')

contents_to_embed = df_feeds['content_for_embedding'].tolist()
embeddings = embedding_model.encode(contents_to_embed, show_progress_bar=True)

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings).astype('float32'))

print(f"\nShape dos embeddings: {embeddings.shape}")
print(f"Total de documentos indexados no FAISS: {index.ntotal}")

In [ ]:
def search_similar_news(query, k=4):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding).astype('float32'), k)
    retrieved_docs = df_feeds.iloc[indices[0]]
    return retrieved_docs

## Configuração do Modelo de Linguagem (LLM) para Geração

In [ ]:
#model_name = "stabilityai/stablelm-zephyr-3b"
model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_compute_dtype": torch.bfloat16,
}

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    trust_remote_code=True,
)


tokenizer = AutoTokenizer.from_pretrained(model_name)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
)

## 	Sistema de Resposta com Geração Aumentada por Recuperação (RAG)

In [ ]:
def generate_answer(query, search_results):
    descriptions = [d[:2000] for d in search_results['descricao'].tolist()]
    context = "\n\n".join(descriptions)

    prompt_template = f"""<|system|>
Você é um assistente de notícias especializado em análise precisa e linguagem sofisticada. Sua tarefa é responder à pergunta do usuário usando APENAS o contexto fornecido. Não invente informações além do contexto. Se a resposta não estiver no contexto, diga: "Com base nas notícias analisadas, não tenho informações suficientes para responder a essa pergunta."

Regras:
1. Use linguagem formal e, quando possível, termos técnicos ou eruditos relacionados ao tema.
2. Seja conciso, mas inclua análise breve quando existirem dados conflitantes.
3. Se houver dados conflitantes, explique brevemente a divergência antes de responder.
4. Sempre indique claramente se a informação é insuficiente para responder.
5. Resuma em 2 ou 3 frases claras e objetivas.
6. Não use linguagem coloquial.

Contexto:
{context}

Pergunta: {query}</s>
<|assistant|>
"""

    response = text_generator(prompt_template)

    return response[0]['generated_text'].split("<|assistant|>")[1].strip()


### Exemplos

In [ ]:

torch.cuda.empty_cache()
gc.collect()

#user_query = "O dolar aumentou ou desceu?"
user_query = "O dólar incidiu em apreciação ou padeceu de depreciação?"
#user_query = "O dólar incorreu em hipertrofia valorativa ou padeceu de depauperação cambial?"
retrieved_news = search_similar_news(user_query, k=5)

print("--- Notícias recuperadas para gerar a resposta: ---")
display(retrieved_news[['titulo', 'fonte']])
print("--------------------------------------------------\n")

final_answer = generate_answer(user_query, retrieved_news)

print(f"Pergunta do Usuário: {user_query}\n")
print(f"Resposta do Chatbot:\n{final_answer}")